In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import plotly.graph_objs as go
from datetime import datetime, timedelta


In [2]:
ticker = 'AAPL'
end_date = datetime.today()
start_date = end_date - timedelta(days=5*365)

data = yf.download(ticker, start=start_date, end=end_date)
data = data[['Close']]


[*********************100%%**********************]  1 of 1 completed


In [3]:
data.tail()

,Close
Date,
2024-06-28,210.619995
2024-07-01,216.750000
2024-07-02,220.270004
2024-07-03,221.550003
2024-07-05,226.339996


In [4]:
data.head()

,Close
Date,
2019-07-08,50.005001
2019-07-09,50.310001
2019-07-10,50.807499
2019-07-11,50.437500
2019-07-12,50.825001


In [5]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

train_data_len = int(np.ceil(len(scaled_data) * 0.8))

train_data = scaled_data[0:train_data_len, :]
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))


In [6]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=25))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(x_train, y_train, batch_size=1, epochs=25)


c:\Users\apurb\OneDrive\Desktop\Streamlite\env\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/25
947/947 ━━━━━━━━━━━━━━━━━━━━ 18s 16ms/step - loss: 0.0084
Epoch 2/25
947/947 ━━━━━━━━━━━━━━━━━━━━ 23s 19ms/step - loss: 0.0011
Epoch 3/25
947/947 ━━━━━━━━━━━━━━━━━━━━ 17s 18ms/step - loss: 7.8462e-04
Epoch 4/25
947/947 ━━━━━━━━━━━━━━━━━━━━ 19s 20ms/step - loss: 6.7194e-04
Epoch 5/25
947/947 ━━━━━━━━━━━━━━━━━━━━ 25s 26ms/step - loss: 5.1723e-04
Epoch 6/25
947/947 ━━━━━━━━━━━━━━━━━━━━ 20s 21ms/step - loss: 6.3864e-04
Epoch 7/25
947/947 ━━━━━━━━━━━━━━━━━━━━ 21s 22ms/step - loss: 4.6590e-04
Epoch 8/25
947/947 ━━━━━━━━━━━━━━━━━━━━ 20s 21ms/step - loss: 4.3357e-04
Epoch 9/25
947/947 ━━━━━━━━━━━━━━━━━━━━ 19s 20ms/step - loss: 5.5464e-04
Epoch 10/25
947/947 ━━━━━━━━━━━━━━━━━━━━ 20s 21ms/step - loss: 4.7429e-04
Epoch 11/25
947/947 ━━━━━━━━━━━━━━━━━━━━ 18s 19ms/step - loss: 3.6360e-04
Epoch 12/25
947/947 ━━━━━━━━━━━━━━━━━━━━ 21s 20ms/step - loss: 4.6371e-04
Epoch 13/25
947/947 ━━━━━━━━━━━━━━━━━━━━ 20s 22ms/step - loss: 4.8542e-04
Epoch 14/25
947/947 ━━━━━━━━━━━━━━━━━━━━ 19s 20ms/step 

In [7]:
test_data = scaled_data[train_data_len - 60:, :]
x_test = []
y_test = data['Close'][train_data_len:].values

for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))


In [8]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)


8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 174ms/step


In [9]:
train = data[:train_data_len]
valid = data[train_data_len:]
valid['Predictions'] = predictions

fig = go.Figure()
fig.add_trace(go.Scatter(x=train.index, y=train['Close'], mode='lines', name='Training Data'))
fig.add_trace(go.Scatter(x=valid.index, y=valid['Close'], mode='lines', name='Actual Price'))
fig.add_trace(go.Scatter(x=valid.index, y=valid['Predictions'], mode='lines', name='Predicted Price'))
fig.update_layout(title='Stock Price Prediction', xaxis_title='Date', yaxis_title='Close Price USD ($)')
fig.show()


C:\Users\apurb\AppData\Local\Temp\ipykernel_12112\1671528379.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['Predictions'] = predictions


In [10]:
last_60_days = scaled_data[-60:]
forecast_input = last_60_days.reshape((1, last_60_days.shape[0], 1))
predicted_prices = []

for _ in range(30):
    prediction = model.predict(forecast_input)
    predicted_prices.append(prediction[0, 0])
    forecast_input = np.append(forecast_input[:, 1:, :], prediction.reshape(1, 1, 1), axis=1)

predicted_prices = scaler.inverse_transform(np.array(predicted_prices).reshape(-1, 1))

future_dates = pd.date_range(end_date, periods=30).tolist()

fig.add_trace(go.Scatter(x=future_dates, y=predicted_prices.flatten(), mode='lines', name='Forecasted Price'))
fig.show()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━